In [1]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [4]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [5]:
def sequential_generator(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=2:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [6]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [7]:
n_labels =2

In [14]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt')]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt')]

In [15]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [16]:
def sequential_generator_germeval(filename, batch_size):
    
    f = open(filename)
    
#     file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
#     shuffled_indexes = range(1, file_length + 1)
    # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#     index_position = 0
    
    

    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    # batch_features_idx = np.zeros((batch_size, nb_sequence_length))
        batch_labels = np.zeros((batch_size, 2))
        # print(len(features))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
#             line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            if len(data)!=3:
                i-=1
                continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
#             index_position += 1
#             if index_position == file_length:
                # shuffle indexes again
#                 shuffled_indexes = range(1, file_length + 1)
                # shuffled_indexes = sample(shuffled_indexes, len(shuffled_indexes))
#                 index_position = 0
#                 break
        # yield [batch_features_ft, batch_features_lg], batch_labels
        yield ([batch_features_ft], batch_labels)

In [12]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model.h5', 
                             monitor='val_acc', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [24]:
no_of_tests = 5

In [ ]:
f = open('/home/jindal/notebooks/jindal/NER/language_model/result_no_pretraining.txt','w')
total_f1 = 0
total_prec =0
total_recall =0
total_acc = 0
for test_no in range(no_of_tests):
    print("test number: %d" %(test_no+1))
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
    model = Model(model_input_embedding, model_output)
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     model.summary()

    # model.reset_states()
    max_f1 = 0
    max_p=0
    max_r=0
    max_a=0
    for epoch in range(epochs):
        print("Epoch: %d" %(epoch+1))
        model.fit_generator(
            sequential_generator_germeval('/home/gwiedemann/notebooks/OffLang/sample_train.txt', batch_size), 
            steps_per_epoch=steps_per_epoch, epochs=1,
        )

        testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
        for i in range(len(dev_sentences)):
            testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
        results = model.predict(testset_features)

        idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
        predLabels = results.argmax(axis=-1)
        devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
        # print(idx2Label)
        # print(predLabels)
        # print(devLabels)
        f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
        r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
        p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
        a = accuracy_score(devLabels, predLabels)
        if max_f1 < f1:
            print("model saved. F1 is %f" %(f1))
            model.save('best_classification_model.h5')
            max_f1 = f1
            max_p = p
            max_a = a
            max_r =r
        text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
        print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
    to_write = "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
    print(to_write)
    print("*****************************************************************************")
    f.write(to_write)
    total_f1+=max_f1
    total_prec+=max_p
    total_acc+=max_a
    total_recall+=max_r
final_text = "avg_prec: " +str(total_prec/no_of_tests)+" total_rec: "+str(total_recall/no_of_tests) +" total_f1: "+str(total_f1/no_of_tests) +" total_acc: "
+str(total_acc/no_of_tests)+" \n"
f.write(final_text)
f.close()

test number: 1
Epoch: 1
Epoch 1/1
132/132 [==============================] - 37s 281ms/step - loss: 0.6277 - acc: 0.6686
model saved. F1 is 0.463007
Test-Data: Prec: 0.638, Rec: 0.363, F1: 0.463, Acc: 0.722
Epoch: 2
Epoch 1/1
132/132 [==============================] - 31s 237ms/step - loss: 0.5298 - acc: 0.7344
model saved. F1 is 0.603306
Test-Data: Prec: 0.673, Rec: 0.547, F1: 0.603, Acc: 0.762
Epoch: 3
Epoch 1/1
132/132 [==============================] - 31s 238ms/step - loss: 0.4846 - acc: 0.7673
model saved. F1 is 0.634731
Test-Data: Prec: 0.679, Rec: 0.596, F1: 0.635, Acc: 0.774
Epoch: 4
Epoch 1/1
132/132 [==============================] - 32s 239ms/step - loss: 0.4551 - acc: 0.7822
model saved. F1 is 0.645291
Test-Data: Prec: 0.694, Rec: 0.603, F1: 0.645, Acc: 0.781
Epoch: 5
Epoch 1/1
132/132 [==============================] - 32s 243ms/step - loss: 0.4263 - acc: 0.7995
model saved. F1 is 0.660118
Test-Data: Prec: 0.694, Rec: 0.629, F1: 0.660, Acc: 0.786
Epoch: 6
Epoch 1/1
132/13